# Réseaux de Neurones Récurrents (RNN & LSTM)

Les réseaux de neurones récurrents sont conçus pour traiter des **données séquentielles** comme le texte, la parole ou les séries temporelles. 
 
Ils exploitent l’ordre des éléments dans une séquence.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS = 10000 
MAX_LEN = 200    
BATCH_SIZE = 64

epochs = 10

## 1. Padding et Séquençage

### Séquençage
Les textes sont convertis en **séquences d’indices** :

"You are ready" ---> [3, 6, 11]

### Padding
Les réseaux exigent des séquences de **longueur identique**.

- Ajout de zéros (`0`) en début ou fin de séquence

- Permet le traitement par batch

**Exemple**

`[3, 6, 11] ---> [0, 3, 6, 11]`

`[34, 56] ---> [0, 0, 34, 56]`

In [ ]:
raw_sentences = [
    [1, 2, 3],
    [4, 5, 6, 7, 8],
    [9]
]

tensors = [torch.LongTensor(s) for s in raw_sentences]

padded_batch = pad_sequence(tensors, batch_first=True, padding_value=0)

print(f"Shape résultante : {padded_batch.shape}")
print("Matrice après padding :\n", padded_batch)

In [ ]:
attention_mask = (padded_batch != 0).long()

print("Masque :\n", attention_mask)

In [ ]:
lengths = torch.tensor([len(s) for s in raw_sentences])

packed_input = pack_padded_sequence(padded_batch, lengths, batch_first=True, enforce_sorted=False)

print("Packed Sequence (Data optimisée) :")
print(packed_input.data) 

*Source code : `embed_seq.py`*

## 2. La Couche d’Embedding (Embedding Layer)

La couche d’**embedding** transforme chaque mot (index entier) en un **vecteur dense** de dimension fixe.

- Apprise pendant l’entraînement

- Capture des relations sémantiques entre mots

- Alternative aux représentations sparse (BoW, TF-IDF)

**Exemple**
- Mot → index → vecteur de dimension 100

In [ ]:
embedding_layer = nn.Embedding(num_embeddings=10, embedding_dim=3)

input_indices = torch.LongTensor([1, 4, 9])

output_vectors = embedding_layer(input_indices)

print(f"Indices d'entrée : {input_indices}")
print(f"Sortie (Shape {output_vectors.shape}) :")
print(output_vectors)

In [ ]:
pretrained_weights = torch.FloatTensor([
    [0, 0, 0],
    [1, 1, 1],
    [2, 2, 2],
    [3, 3, 3],
    [4, 4, 4]
])

pretrained_layer = nn.Embedding.from_pretrained(pretrained_weights)

input_test = torch.LongTensor([2, 4])
print("\n--- Expérience 2 : Poids Pré-entraînés ---")
print(f"Entrée : {input_test}")
print(f"Sortie :\n{pretrained_layer(input_test)}")

In [ ]:
pretrained_layer.weight.requires_grad = False

print("\n--- Expérience 3 : Vérification du Gel ---")
print(f"La couche apprend-elle ? -> {pretrained_layer.weight.requires_grad}")

*Source code : `emb_seq.py`*

## 3. Réseaux de Neurones Récurrents (RNN)

Un **RNN** traite les données mot par mot (ou token par token) en conservant une **mémoire** de ce qui a été vu précédemment.

- La sortie dépend de l’entrée courante et de l’état précédent

- Adapté aux séquences de longueur variable

**Limites**

- Difficulté à apprendre des dépendances longues

- Problème de gradient qui disparaît (vanishing gradient)

In [ ]:
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=MAX_WORDS)
print(f"Train samples: {len(input_train)}")
print(f"Exemple d'avis (sous forme d'index): {input_train[0][:10]}...")

# mettre les données à la même echelle
input_train = pad_sequences(input_train, maxlen=MAX_LEN)
input_test = pad_sequences(input_test, maxlen=MAX_LEN)
print(f"Shape des données d'entrée : {input_train.shape}")

In [ ]:
class IMDbDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = torch.LongTensor(sequences)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

train_dataset = IMDbDataset(input_train, y_train)
test_dataset = IMDbDataset(input_test, y_test)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size]
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size=10000, embedding_dim=32, hidden_dim=32):
        super(SentimentRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        embedded = self.embedding(x)
        _, h_n = self.rnn(embedded)
        h_n = h_n.squeeze(0) 
        out = self.fc(h_n)
        out = self.sigmoid(out)
        return out.squeeze(1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUtilisation de : {device}")

model = SentimentRNN().to(device)

print("\nArchitecture du modèle :")
print(model)
print(f"\nNombre de paramètres : {sum(p.numel() for p in model.parameters()):,}")

criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for sequences, labels in loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(loader), correct / total


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for sequences, labels in loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    return total_loss / len(loader), correct / total

In [ ]:
for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2%}")
    print(f"  Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2%}")

test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"\nAccuracy sur le Test Set : {test_acc:.2%}")

*Source code : `RNN.py`*

## 4. LSTM et GRU

### LSTM (Long Short-Term Memory)

Les **LSTM** améliorent les RNN classiques grâce à des **portes** (gates) qui contrôlent l’information.

- Porte d’oubli

- Porte d’entrée

- Porte de sortie

**Avantages**

- Capture des dépendances à long terme

- Très utilisé en NLP et séries temporelles

---

### GRU (Gated Recurrent Unit)

Les **GRU** sont une version simplifiée des LSTM.

- Moins de paramètres

- Entraînement plus rapide

- Performances souvent comparables aux LSTM

---

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_WORDS)

x_train = pad_sequences(x_train, maxlen=MAX_LEN)
x_test = pad_sequences(x_test, maxlen=MAX_LEN)

train_data = TensorDataset(torch.from_numpy(x_train).long(), torch.from_numpy(y_train).float())
test_data = TensorDataset(torch.from_numpy(x_test).long(), torch.from_numpy(y_test).float())

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text):
        embedded = self.embedding(text)
        
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        last_hidden = hidden[-1]
        
        return self.sigmoid(self.fc(last_hidden))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(vocab_size=MAX_WORDS, embedding_dim=32, hidden_dim=64, output_dim=1)
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(f"\nEntraînement sur {device}...")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions = model(inputs).squeeze()
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\nAccuracy Finale : {100 * correct / total:.2f}%")

*Source code : `LSTM.py`*

## Résumé

| Élément | Rôle |
|------|-----|
| Embedding | Représentation dense des mots |
| Padding | Uniformisation des séquences |
| Séquençage | Conversion texte → indices |
| RNN | Modélisation séquentielle |
| LSTM | Mémoire longue durée |
| GRU | LSTM simplifié |